In [0]:


# Add EntityPath to your existing connection string
secret_connection = dbutils.secrets.get(scope="weather-secrets", key="eventhub-connection")
event_hub_name = "weather-hub"  # ← Your actual event hub name

if "EntityPath=" not in current_connection:
    if current_connection.endswith(";"):
        connection_string = f"{current_connection}EntityPath={event_hub_name}"
    else:
        connection_string = f"{current_connection};EntityPath={event_hub_name}"
    
    print(f"✅ Added EntityPath. New string: {connection_string[:60]}...")

In [0]:
# Debug: Check connection string
print(f"Connection string length: {len(connection_string)}")
print(f"First 50 chars: {connection_string[:50]}...")
print(f"Contains 'EntityPath=': {'EntityPath=' in connection_string}")

# Test encryption
try:
    encrypted = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connection_string)
    print("✅ Encryption successful!")
except Exception as e:
    print(f"❌ Encryption failed: {e}")

In [0]:
# COMMAND ----------
from pyspark.sql.functions import col, current_timestamp

# 1. Get connection string
scope_name = "weather-secrets"
secret_key = "ev-connection"
connection_string = dbutils.secrets.get(scope=scope_name, key=secret_key)

# 2. FIX: Ensure EntityPath is included
event_hub_name = "weather-raw"  # ← REPLACE WITH YOUR ACTUAL EVENT HUB NAME
if "EntityPath=" not in connection_string:
    if connection_string.endswith(";"):
        connection_string = f"{connection_string}EntityPath={event_hub_name}"
    else:
        connection_string = f"{connection_string};EntityPath={event_hub_name}"
    print(f"✅ Added EntityPath to connection string")

# 3. Configure Event Hubs WITH ENCRYPTION
eh_conf = {
    'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connection_string)
}

# 4. Read the stream from Event Hubs
df_raw = (spark.readStream
    .format("eventhubs")
    .options(**eh_conf)
    .load())

# 5. Transform to Bronze
df_bronze = df_raw.select(
    col("body").cast("string").alias("raw_payload"),
    current_timestamp().alias("ingested_at"),
    col("enqueuedTime").alias("event_hub_time")
)

# 6. Write to Bronze Delta Table
(df_bronze.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", "/mnt/weather/checkpoints/bronze")
    .toTable("weather_bronze_raw"))

In [0]:
# Check if the Java class is visible
try:
    jvm_check = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.getClass().getName()
    print(f"✅ Library is ACTIVE: {jvm_check}")
except Exception as e:
    print("❌ Library still not visible. Please ensure it's 'Installed' on the 'Libraries' tab of the cluster.")

In [0]:
# Check the last 10 rows that arrived
display(spark.read.table("weather_bronze_raw").orderBy(col("ingested_at").desc()).limit(10))